# Part 1

You receive a signal directly from the CPU. Because of your recent assistance with jump instructions, it would like you to compute the result of a series of unusual register instructions.

Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0. The instructions look like this:

```
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
```

These instructions would be processed as follows:

- Because a starts at 0, it is not greater than 1, and so b is not modified.
- a is increased by 1 (to 1) because b is less than 5 (it is 0).
- c is decreased by -10 (to 10) because a is now greater than or equal to 1 (it is 1).
- c is increased by -20 (to -10) because c is equal to 10.
- After this process, the largest value in any register is 1.

You might also encounter `<=` (less than or equal to) or `!=` (not equal to). However, the CPU doesn't have the bandwidth to tell you what all the registers are named, and leaves that to you to determine.

What is the largest value in any register after completing the instructions in your puzzle input?

In [4]:
import operator as op
from collections import defaultdict

from dataclasses import dataclass

In [8]:
class Instruction:
    _op_map = {
        'inc': op.add,
        'dec': op.sub
    }
    _cmp_map = {
        '>': op.gt,
        '>=': op.ge,
        '<': op.lt,
        '<=': op.le,
        '==': op.eq,
        '!=': op.ne
    }
    
    def __init__(self, instr_str):
        name, op_name, delta_str, _, cmp_target, cmp_name, cmp_val_str = instr_str.strip().split()
        self.delta_target = name
        self.op = self._op_map[op_name]
        self.delta_val = int(delta_str)
        self.cmp_target = cmp_target
        self.cmp = self._cmp_map[cmp_name]
        self.cmp_val = int(cmp_val_str)
    
    def cmp_op(self, val):
        return self.cmp(val, self.cmp_val)

In [9]:
def pzl_input(input_file='./inputs/day08/input.txt'):
    with open(input_file) as f:
        yield from (Instruction(line) for line in f)

def test_input():
    yield from pzl_input('./inputs/day08/test_input.txt')

In [19]:
class Register:
    def __init__(self):
        self.data = defaultdict(lambda: 0)
        self.max_seen = float('-inf')
    
    def process_instr(self, instr: Instruction):
        if instr.cmp_op(self.data[instr.cmp_target]):
            self.data[instr.delta_target] = instr.op(self.data[instr.delta_target], instr.delta_val)
            if self.data[instr.delta_target] > self.max_seen:
                self.max_seen = self.data[instr.delta_target]
    
    def process_instructions(self, instructions):
        for instr in instructions:
            self.process_instr(instr)
    
    def max(self) -> int:
        return max(self.data.values())

In [20]:
def max_register_val(instructions):
    reg = Register()
    reg.process_instructions(instructions)
    return reg.max()

In [21]:
assert max_register_val(test_input()) == 1

In [22]:
max_register_val(pzl_input())

6828

# Part 2

To be safe, the CPU also needs to know the highest value held in any register during this process so that it can decide how much memory to allocate to these operations. For example, in the above instructions, the highest value ever held was 10 (in register c after the third instruction was evaluated).

In [23]:
def max_value_seen(instructions):
    reg = Register()
    reg.process_instructions(instructions)
    return reg.max_seen

In [24]:
assert max_value_seen(test_input()) == 10

In [25]:
max_value_seen(pzl_input())

7234